# **Resolución de la ecuación de Schrödinger unidimensional para un potencial cuadrado. Estudio del coeficiente de transmisión.**
## Problema Voluntario 3
### Física Computacional - 2025

---

**Irene Blanco Ramos**

**Fecha:** 11 de Junio de 2025

---


# Índice

1. [Introducción](#Introduccion)
2. [Resolución de la Ecuación de Schrödinger](#Resolucion-de-la-Ecuacion-de-Schrodinger)
3. [Dependencia del coeficiente de transmisión con los parámetros de la simulación](#Dependencia-del-coeficiente-de-transmision-con-los-parametros-de-la-simulacion)
4. [Cálculo de los valores esperados de observables](#Calculo-de-los-valores-esperados-de-observables)
5. [Potencial multibarrera](#Potencial-multibarrera)
6. [Optimización y paralelización](#Optimizacion-y-paralelizacion)

---

# Introducción 

El objetivo de este informe es estudiar el coeficiente de transmisión de una partícula cuántica que atraviesa una barrera de potencial cuadrado de altura finita. Para ello, se ha realizado una simulación en C que resulve numéricamente la ecuación de Schrodinger unidimensional y se ha calculado el coeficiente de transmisión en diferentes casos imponiendo una función de onda inicial gaussiana para la partícula. 

En sistemas clásicos, una partícula es capaz de atravesar una barrera de energía potencial $V_0$ siempre que su energía cinética $E$ sea mayor que esta altura del potencial. Es caso contrario, la partícula rebotaría cuando $E<V_0$. Es decir, que la probabilidad de que la partícula traspase la barrera de potencial va a ser 0 o 1. 

<p align="center">
  <img src="efectotunel.jpg" alt="Etunel" width="300">
</p>

Sin embargo, en mecánica cuántica se pone de manifiesto el efecto túnel debido a la naturaleza ondulatoria de la materia. Cada partícula lleva asociada una función de onda de forma que existe una probabilidad no nula de que se atraviese la barrera cuando $E<V_0$. La función de onda decae exponencialmente al entrar en la barrera pero, como tiene una anchura finita, la función de onda no se anula al otro lado por lo que una parte es transmitida. Además, cuando la energía de la partícula es $E>V_0$, la partícula tiene suficiente energía como para atravesar la barrera aunque también hay una probabilidad de que sea reflejada. Esto se cuantifica con los coeficientes de reflexión y transmisión, que pueden oscilar entre cero y uno. En el informe nos centraremos en el coeficiente de transmisión $K$.

---

# Resolución de la Ecuación de Schrödinger

Para resolver la ecuación de Schrodinger numéricamente se ha utilizado un método de discretización de la ecuación mediante diferencias finitas. Se crea una malla unidimensional (en la dirección $x$) discretizando tanto el dominio espacial como el temporal. Previamente se ha realizado un reescalamiento de la masa $m=1/2$ de la partícula y la constante de Planck $\hbar=1$ para simplificar las ecuaciones a resolver. 

Se ha dividido el espacio en $N$ puntos con un paso de $h$ que se ha elegido como $h=0.01$ para todas las simulaciones. El tiempo se ha dividido en $T$ pasos que representan el tiempo total de evolución de la función de onda. Para cada instante de tiempo en una posición concreta $x=jh$, el algoritmo resuelve un sitema de matrices tridiagonales para calcular la correspondiente función de onda. Se parte inicialmente de un paqute de ondas gaussiano
$$
\Phi(x, 0) = e^{ik_0x}e^{-(x-x_0)^2/2\sigma^2}
$$

centrado en el punto $x=Nh/4$ y de anchura $\sigma= Nh/16$.
La anchura de la barrera de potencial, centrado en $N/2$, ha sido $N/5$ y su altura depende de la energía de la función de onda incidente $\lambda k_0^2$, donde $\lambda=\frac{E}{V_0}$.

El parámetro $k_0$ es reescalado a $\tilde{k}_0 = k_0 h = 2\pi n_{ciclos}/N$ donde se ha mantenido  $n_{ciclos} = N/16$ en cada simulación. Esta  relación entre $k_0$ y $n_{ciclos}$ refleja el número de oscilaciones completas que la función de onda realiza en la red.

$$\tilde{V}_j = V_j h^2 = \begin{cases} 0 & \text{si } j \notin [2N/5, 3N/5] \\ \lambda \tilde{k}_0^2 & \text{si } j \in [2N/5, 3N/5] \end{cases}$$

---

# Dependencia del coeficiente de transmisión con los parámetros de la simulación
## Coeficiente de transmisión y probabilidad máxima
Para calcular el coeficiente de transmisión $K$, se ha calculado la probabilidad de encontrar la partícula, que incide desde la izquierda en el potencial, a la derecha de la barrera. Para conseguirlo se ha colocado un detector al final de la barrera de un ancho de $N/5$ tal que la probabilidad de encontrar la partícula en esa región se calcula como:
$$
P_D(n) = \sum_{j=4N/5}^{N} |\Phi_{j,n}|^2.
$$
donde $\Phi_{j,n}$ es la función de onda en la posición $j<N$ en el intante $n<T$. 

Para determinar el tiempo necesario de evolución de la función de onda antes de tomar una medida con el detector, se deja evolucionar primero la función de onda el tiempo $T$ calculando la probabilidad en cada instante $n$. Después, se calcula el instante $n_D$ en el que la probabilidad $P_D$ sea máxima, dejando ahora evolucionar la función de onda $n_D$ pasos de tiempo. Esto se repite para $m$ repeticiones, de manera que en cada repetición se compare $P_D(n_D)$ con un número $p$ aleatorio entre 0 y 1. Cuando $p<P_D(n_D)$ la partícula se habrá detectado a la derecha y se actualiza un contador $m_T$. Si $p>P_D(n_D)$, la partícula no se ha detectado y el contador no suma +1. Finalmete, el coeficiente de transmisión, vendrá dado por el número de medidas detectadas $m_T$ entre el número de medidas totales $m$:
$$
K=\frac{m_T}{m}
$$

Maximizar $P_D(n)$ tiene sentido porque la función de onda se dispersa y parte de la probabilidad puede reflejarse o transmitirse en distintos instantes. Cuando calculamos el máximo, estamos garantizando que tomamos la medida cuando la mayor parte de la función de onda posible ha llegado al detector, evitando infravalorar el valor.

## Coeficiente de transmisión y altura del potencial $\lambda$

Es posible resolver este problema analíticamente, obteniendo una expresión teórica del coeficiente de transmisión. Para obtener esta expresión, se resuelve la ecuación de Schrödinger antes, dentro y fuera del potencial aplicando condiciones de contorno de continuidad en la función de onda y su derivada. Para un potencial cuadrado de altura $V_0$ y anchura $a$, el coeficiente de transmisión para una partícula de energía $E$ y número de onda incidente $K_0$ es:

-   Para $E < V_0$:     $K = \frac{4E(V_0-E)}{4E(V_0-E)+V_0^2 \sinh^2(k_0 a)}$

-  Para $E>V_0$:     $K = \frac{4E(E-V_0)}{4E(E-V_0)+V_0^2 \sinh^2(k_0 a)}$

En nuestra simulación, $E$ y $V_0$ se relacionan mediante el parámetro $\lambda$. Al aumentar $\lambda$, la energía del potencial incrementa mientras que la energía inicial de la partícula $E$ se mantiene constante. De este modo, en la simulación podemos observar cómo influye la altura de la barrera en la transmisión de una misma partícula (con la misma energía). Para estudiar esto, hemos expresado el valor teórico de $K$ en función de los parámetros de la simulación mencionados en el apartado anterior, llegando a que:

- Para $\lambda < 1$:     $K= \frac{4(1 - \lambda)}{4(1 - \lambda) + \lambda^2 \text{sen}^2 \left(\frac{2\pi}{5} n_{ciclos} \sqrt{1 - \lambda}\right)}$
- Para $\lambda > 1$:  $K= \frac{4(\lambda - 1)}{4(\lambda - 1) + \lambda^2 \text{senh}^2 \left(\frac{2\pi}{5} \sqrt{\lambda - 1}\right)}$

Representando estas funciones gráficamente, se obtiene que:
<p align="center">
  <img src="Kteorico.png" alt="Kteorico">
</p>

En la imagen vemos una división clara entre la zona $\lambda < 1$ (morado) y $\lambda > 1$ (naranja). A partir de $\lambda=1$ se muestra una transición abrupta que indica que para grandes valores de la altura del potencial la partícula prácticamente no se transmite. El coeficente $K$ es nulo en este caso, por lo que para una altura elevada de la barrera la patícula se comporta clasicamente (rebota). 

Ampliando los valores de $K$ entre 0 y 1, se observa también que el coeficiente de transmisión va disminuyendo, convergiendo a cero, pero también aparecen oscilaciones que reflejan el término sinusoidal de la expersión. Estas oscilaciones, llamadas resonancias de transmisión, representan los fenómenos de interferencias que ocurren cuando la partícula interactúa con la barrera. Cuando la anchura del potencial es un múltiplo entero de la semilongitud de onda, la partícula lo atraviesa con una probabilidad alta. Mientras que si se produce una interferencia destructiva la probabilidad de transmisión es mucho menor. Tal y como hemos expresado $K$ el término del seno en el denominador depende del número de ciclos $n_{ciclos}$.

La siguiente gráfica, muestra los resultados obtenidos con la simulación:

<p align="center">
  <img src="Kexperimental.png" alt="Kexperimental">
</p>


Aunque no se observan las oscilaciones de resonancia entre 0 y 1, vemos que la tendencia de $K$ a decaer con $\lambda $ se cumple con una diferencia brusca en $\lambda=1$, que es el punto crítico en el que la energía de la partícula es igual a la del potencial. A partir de ese punto, la barrera de potencial supera a la de la partícula y ésta casi no se transmite. Para ilustrar este fenómeno, se adjuntan a continuación dos animaciones para $\lambda=0.5$ y $\lambda=5.0$. En la primera, la onda traspasa la barrera sin problemas mientras que en la segunda casi toda la onda se refleja al llegar a la barrera y se transmite una fracción muy pequeña.

<p align="center">
  <video src="lambda05.mp4" width="400" controls></video>
  <video src="lambda5.mp4" width="400" controls></video>
</p>

## Coeficiente de transmisión y parámetro espacial $N$

En la figura, se presentan los resultados de $K$ en función del número de pasos espaciales $N$. Se ha tenido en cuenta que al aumentar el número de puntos espaciales, la onda necesitaba más puntos temporales para llegar a al otro lado de la barrera y ser medida por el detector. Por lo que al aumentar $N$ (sobre todo para $N$ muy grandes), se ha aumentado también prograsivamente $T$. Además, el tiempo se ha regulado de manera que la primera repetición que es la que se ha dejado evolucionar un tiempo $T$ al completo termine justo antes de que la onda se refleje al atravesar la barrera, ya que esto daba problemas con el cálculo de la probabilidad máxima. El número de repeticiones o experimentos ha sido 400, lo que equivale a tomar 400 medidas.

<p align="center">
  <img src="KvsNgrafico.png" alt="KvsNgrafico">
</p>


En la gráfica, el coeficiente de transmisión se ha representado para dos alturas diferentes $\lambda=0.3$ y $\lambda=1.0$. En ambos casos vemos que el valor de $K$ es casi constante y se estabiliza para altos valores de $N$. Se ha estimado un valor aproximado de $K=0.94$ para $\lambda=0.3$ y de $K=0.0$ para $\lambda=1.0$. 

---

# Cálculo de los valores esperados de observables
Para calcular el valor esperado de un observable $A$ se utiliza:
$$
\langle A \rangle = \int \phi^*(x) \hat{A} \phi(x) dx
$$
donde $\hat{A}$ es el operador del observable correspondiente.  
En Física cuática la incertidumbre de una medida procede del origen probabilístico de medir un observable y el principio de indertidumbre de Heisenberg. La incertidumbre de un observable está dada por la desviación de su valor medido con respecto al valor esperado, por lo que se calcula como:
$$
\Delta A = \sqrt{\langle \hat{A}^2 \rangle - (\langle \hat{A} \rangle)^2}
$$
Con esta expresión se han calculado las barras de error de las gráficas que representan la evolución temporal de los valores esperados de la posición y de la energía cinética, que se muestran a continuación. Cada observable se ha representado para dos alturas de potencial diferentes $\lambda=0.3$ y $\lambda=1.0$. 
## Valor esperado de la posición y la energía cinética

Para el valor esperado de la posición y su incertidumbre, se ha empleado la expresión anterior sustituyendo el operador posición $\hat{x} = x$, considerando que $x=jh$.



Para calcular el valor esperado de la energía cinética se ha utilizado el operador momento $\hat{p} = -i\hbar \frac{\partial}{\partial x}$, ya que la energía cinética es:

$$ \hat{E} = \frac{\hat{p}^2}{2m}= -\frac{1}{2} \frac{d^2}{dx^2} $$

Para resolver esta segunda derivada (Laplaciano) numéricamente se ha implementado en el programa una función en la que calcula la segunda derivada discreta de la función de onda por diferencias finitas. El código consiste en calcular el laplaciano en cada punto usando sus vecinos inmediatos. En el código:

$$\frac{d^2\phi_{1,j}}{dx^2}=\frac{\phi_{1,j+1} - 2.0\phi_{1,j} + \phi_{1,j-1}} {h^2}$$

Después, se multiplica por el complejo conjugado de la función de onda y sumar sobre todos los puntos, multiplicando al final por el factor $-0.5$. 

Para determinar $\Delta E$ se ha calculado el valor esperado de la energía al cuadrado $\langle E^2 \rangle$. Ahora el operador está al cuadrado por lo que se ha calculado la cuarta derivada discreta de la función de onda, que corresponde a aplicar dos veces el operador (la segunda derivada). El algoritmo uitlizado para aproximar la cuarta derivada central por diferencias finitas ha sido:

$$\frac{d^4\phi_{1,j}}{dx^4} = \frac{\phi_{1,j-2} - 4.0\phi_{1,j-1} + 6.0\phi_{1,j} - 4.0\phi_{1,j+1} + \phi_{1,j+2}} { h^4}$$

De forma análoga, se ha multiplicado por el conjugado de la función de onda y se ha sumado sobre todos los puntos multiplicando el resultado final por $-0.25$, ya que  $( -\frac{1}{2} \frac{d^2}{dx^2} )^2 = \frac{1}{4} \frac{d^4}{dx^4}$.

La gráfica muestra cómo varían con el tiempo el valor esperado de la posición $\langle x \rangle$ y de la energía cinética $\langle E \rangle$ de la partícula.

<p align="center">
  <img src="graficoxTesperados.png" alt="xTesperados">
</p>

En la figura vemos que la posición presenta un máximo y un mínimo en los instantes en los que la energía cinética se estabiliza, mientras que la energía cinética se hace mínima cuando la posición esperada crece o decrece. Los mínimos de la energía cinética tienen lugar cuando la partícula interactúa con la barrera de potencial. Por conservación de la energía, cuando la partícula está traspasando adquiere parte de la energía potencial y la energía cinética se minimiza. El la representación de la posición en función del tiempo, se observa que cuando la partícula atraviesa el potencial se refleja, causando el cambio de pendiente de la gráfica.

Si la altura del potencial fuese mayor, la fracción de la onda transmitida sería menor, por lo que se observaría un pico en el valor esperado al llegar al potencial, ya que una parte significativa de la onda se reflejaría al interactuar con el mismo. En ese caso, el primer mínimo de energía cinética coincidiría con el máximo del valor esperado.

---

# Potencial multibarrera

Por último, se ha estudiado el coeficiente de transmisión del paquete de ondas aumentando el número de barreras $n$ equiespaciadas para un valor fijo $\lambda=0.5$ y 400 repeticiones. Para determinar la dependencia entre $K$ y $n$ se ha realizado un ajuste exponencial como se muestra a continuación.

<p align="center">
  <img src="Kbarrerasgrafico.png" alt="Kbarreras">
</p>

Se ha calculado el coeficiente de Pearson para comprobar la validez del ajuste. Como su valor es muy cercano a la unidad, podemos decir que nuestros datos se ajustan a una función exponencial. Por tanto, al aumentar el número de barreras el coeficiente de transmisión decae exponencialmente.

Estas animaciones ilustran la evolución de la función de onda para 5 y 10 barreras con una anchura de 200 y una separación también de 200 para $\lambda=0.5$. Aunque en ambos casos se transmite la onda, vemos las múltiples reflexiones que se producen en el segundo caso cuando la onda interactúa con cada barrera.

<p align="center">
  <video src="multibarrera1.mp4" width="400" controls></video>
  <video src="multibarrera2.mp4" width="400" controls></video>
</p>

---

# Optimización y paralelización

El problema de la multibarrera ha sido el que más ha tardado en compilar en el PC utilizado (Windows 11, Procesador	11th Gen Intel(R) Core(TM) i5-1135G7 @ 2.40GHz, 2419 Mhz, 4 procesadores principales, 8 procesadores lógicos). Se ha inetentado compilar el progrma en el cluster Joel, pero ha habido problemas a la hora de ejecutar el programa (tanto para una como para varias barreras) que no han podido ser solucionados por falta de tiempo disponible. Se podría haber estudiado el coste computacional en función del número de barreras con la opciones -O1 -O2 -O3 en el PC pero no ha sido posible por falta de tiempo, aunque si se ha utilizado la función ctime para comprobar el tiempo de ejecución el programa de la multibarrera. Tampoco ha sido posible paralelizar los bucles del programa utilizando la función pragma de OpenMP por el mismo motivo.

---

# Conclusiones
En resumen, en este trabajo se ha analizado el coeficiente de transmisión de una partícula cuántica al atravesar una barrera de potencial mediante una simulación en C. Se ha comprobado cómo influye la altura del potencial en la probabilidad de transmisión, observando el efecto túnel. Los resultados muestran que, para alturas de potencial menores que la energía de la partícula, la transmisión es elevada, mientras que para alturas superiores la transmisión decae rápidamente, reproduciendo el comportamiento clásico en el límite, lo que concuerda con lo esperado teóricamente.
Además, se han calculado los valores esperados de la posición y la energía cinética, donde se ha observado cómo la partícula interactúa con el potencial minimizando su energía cinética. Finalmente, se ha estudiado la reacción de la onda ante un potencial multibarrera, observando una caída exponencial del coeficiente de transmisión al aumentar el número de barreras.

---

# Referencias

1. [ El efecto túnel de las partículas a través de las barreras de potencial](https://openstax.org/books/f%C3%ADsica-universitaria-volumen-3/pages/7-6-el-efecto-tunel-de-las-particulas-a-traves-de-las-barreras-de-potencial)
2. [Imagen de la introducción. Barrera de potencial](https://www.fisicacuantica.es/barrera-de-potencial/)

